# NDIS Database Preparation

## Geohazard & Infrastructure database compilation

This notebook documents the preprocessing pipeline for the Natural Disaster Information System (NDIS), a decision-support tool for planning RPAS (Remotely Piloted Aircraft Systems) missions in geohazard monitoring.

The preprocessing integrates six core datasets, each representing a distinct geohazard type:
1. Volcano
2. Landslide
3. Tsunami
4. Fault
5. Earthquake
6. Nuclear Power Plant

To ensure consistency across hazard types, raw datasets were parsed and harmonized into a unified schema. This involved:
- Converting spatial geometries to a common coordinate system
- Standardizing key fields such as intensity, duration, and economic loss
- Mapping hazard-specific attributes (e.g., magnitude, VEI) into shared analytical fields
- Handling missing values using severity-based proxies or simulation logic

Geospatial preprocessing steps include:
- Buffering and clipping to Exclusive Economic Zone (EEZ) boundaries
- Zonal statistics for population exposure within hazard zones
- Distance-based feature engineering (e.g., proximity to roads or coastlines)

All spatial analysis was performed in **ArcGIS Pro**, which also supported early-stage logic testing and scenario validation. The final decision logic, including RPAS and sensor recommendations, was implemented using a staggered rule-based workflow. Although machine learning models (e.g., decision trees) were evaluated, they were not used in the operational version due to limited generalizability.

This notebook serves as a reproducible reference for harmonized data preparation and logic integration in the NDIS system.

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
%matplotlib inline
#ArcGIS packages
import arcpy
#from arcgis.mapping import WebScene
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from IPython.display import display
from arcgis.features import GeoAccessor
from arcgis import *

# basic packages
import csv
import numpy as np
import os
import timeit
import random
import string
from playsound import playsound

# Data management
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point  # to get points from long lat

# Request service
#from requests import Request
import json
import re
from functools import reduce
#from owslib.wfs import WebFeatureService

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

# Get Datasets

## <font color='red'> 1. Volcano data </font>

<div class="alert alert-block alert-info">
<b>Note:</b>
    Citation: Global Volcanism Program, 2013. Volcanoes of the World, v. 4.11.0 (08 Jul 2022). Venzke, E (ed.). Smithsonian Institution. Downloaded 13 Jul 2022. https://doi.org/10.5479/si.GVP.VOTW4-2013.

Further info: https://volcano.si.edu/database/webservices.cfm

Service Layer: http://webservices.volcano.si.edu/geoserver/GVP-VOTW/ows?service=WFS&version=1.0.0&request=describefeaturetype&typeName=GVP-VOTW:E3WebApp_HoloceneVolcanoes
    
    Significant Volcano Eruption: 
    Citation: National Geophysical Data Center / World Data Service (NGDC/WDS): Significant Earthquake Database. National Geophysical Data Center, NOAA. doi:10.7289/V5TD9V7K
</div>

In [8]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # Update the path accordingly

# Set the workspace to your geodatabase
arcpy.env.workspace = gdb_path

# Specify the feature class name
feature_class_name = "volc84"
feature_class_path = f"{gdb_path}\\{feature_class_name}"

# Describe the feature class
desc = arcpy.Describe(feature_class_path)

# Check the spatial reference
spatial_reference = desc.spatialReference

# Print the spatial reference details
print(f"Spatial Reference of {feature_class_name}:")
print(f"  Name: {spatial_reference.name}")
print(f"  WKID: {spatial_reference.factoryCode}")
print(f"  WKT: {spatial_reference.exportToString()}")

Spatial Reference of volc84:
  Name: GCS_WGS_1984
  WKID: 4326
  WKT: GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision


In [9]:
# Use arcpy to create a list of fields
fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{feature_class_name}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{feature_class_name}", fields) as cursor:
    for row in cursor:
        data.append(dict(zip(fields, row)))

# Convert the list of dictionaries into a DataFrame
volc = pd.DataFrame(data)
volc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OBJECTID          1324 non-null   int64  
 1   Shape             1324 non-null   object 
 2   GmlID             1324 non-null   object 
 3   VolcanoNumber     1324 non-null   int64  
 4   VolcanoName       1324 non-null   object 
 5   Country           1324 non-null   object 
 6   Remarks           1324 non-null   object 
 7   VolcanoType       1324 non-null   object 
 8   LastEruption      857 non-null    float64
 9   Elevation         1324 non-null   int64  
 10  TectonicSetting   1319 non-null   object 
 11  Within_5km        1291 non-null   float64
 12  Within_10km       1291 non-null   float64
 13  Within_30km       1291 non-null   float64
 14  Within_100km      1291 non-null   float64
 15  VPImageFileName   1252 non-null   object 
 16  VPImageCaption    1252 non-null   object 


In [10]:
# Assuming 'Shape' column contains tuples (x, y), get the geometry for volcano database
volc['geometry'] = volc['Shape'].apply(lambda geom: Point(geom) if geom is not None else None)

In [13]:
# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
#volc.set_crs(epsg=4326, inplace=True)
#print(volc.crs)

In [14]:
# Create a GeoDataFrame
volc = gpd.GeoDataFrame(volc, geometry='geometry')
volc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   OBJECTID          1324 non-null   int64   
 1   Shape             1324 non-null   object  
 2   GmlID             1324 non-null   object  
 3   VolcanoNumber     1324 non-null   int64   
 4   VolcanoName       1324 non-null   object  
 5   Country           1324 non-null   object  
 6   Remarks           1324 non-null   object  
 7   VolcanoType       1324 non-null   object  
 8   LastEruption      857 non-null    float64 
 9   Elevation         1324 non-null   int64   
 10  TectonicSetting   1319 non-null   object  
 11  Within_5km        1291 non-null   float64 
 12  Within_10km       1291 non-null   float64 
 13  Within_30km       1291 non-null   float64 
 14  Within_100km      1291 non-null   float64 
 15  VPImageFileName   1252 non-null   object  
 16  VPImageCaption  

In [15]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
vo_df = volc[['VolcanoNumber','LatitudeDecimal','LongitudeDecimal','geometry']].copy()
vo_df.rename(columns = {'VolcanoNumber':'HazardID','LatitudeDecimal':'latitude','LongitudeDecimal':'longitude'}, inplace = True)
vo_df['HazardType'] = 1
vo_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   HazardID    1324 non-null   int64   
 1   latitude    1324 non-null   float64 
 2   longitude   1324 non-null   float64 
 3   geometry    1324 non-null   geometry
 4   HazardType  1324 non-null   int64   
dtypes: float64(2), geometry(1), int64(2)
memory usage: 51.8 KB


## <font color='red'> 2. Landslide Data </font>
Retrieved from NASA

Title: Global Landslide Catalog | Type: Feature Service | Owner: krolikie@unhcr.org_unhcr
https://maps.nccs.nasa.gov/arcgis/apps/MapAndAppGallery/index.html?appid=574f26408683485799d02e857e5d9521
<div class="alert alert-block alert-info">
<b>Note:</b>
    Citation: Kirschbaum, D.B., Stanley, T., & Zhou, Y. (2015). Spatial and temporal analysis of a global landslide catalog. Geomorphology, 249, 4-15. doi:10.1016/j.geomorph.2015.03.016

    Kirschbaum, D.B., Adler, R., Hong, Y., Hill, S., & Lerner-Lam, A. (2010). A global landslide catalog for hazard applications: method, results, and limitations. Natural Hazards, 52, 561-575. doi:10.1007/s11069-009-9401-4

Further info: 
https://gpm.nasa.gov/landslides/data.html
</div>

In [19]:
# After the file is downloaded, load the following from local path and check the GCS
ls_spatial_ref = arcpy.Describe(r"D:/NDIS_Database/04_Landslide/nasa_coolr_reports_point.shp").spatialReference
ls_spatial_ref

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [20]:
# Read the shapefile from local disk
landslide_df = gpd.read_file(r"D:/NDIS_Database/04_Landslide/nasa_coolr_reports_point.shp")
landslide_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   src_name    14718 non-null  object  
 1   src_link    13879 non-null  object  
 2   ev_date     13087 non-null  object  
 3   ev_time     6555 non-null   object  
 4   ev_title    14026 non-null  object  
 5   ev_desc     12176 non-null  object  
 6   loc_desc    13195 non-null  object  
 7   loc_acc     14693 non-null  object  
 8   ls_cat      14720 non-null  object  
 9   ls_trig     14720 non-null  object  
 10  ls_size     14719 non-null  object  
 11  ls_setting  14713 non-null  object  
 12  fatalities  14723 non-null  int64   
 13  injuries    14723 non-null  int64   
 14  storm_name  666 non-null    object  
 15  photo_link  2228 non-null   object  
 16  comments    2560 non-null   object  
 17  ev_imp_src  14723 non-null  object  
 18  ev_imp_id   11689 non-null  object  
 

In [24]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
ls_df = landslide_df[['ev_id','latitude','longitude', 'geometry']].copy()
ls_df.rename(columns = {'ev_id':'HazardID'}, inplace = True)
ls_df['HazardType'] = 2
ls_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   HazardID    14723 non-null  int64   
 1   latitude    14723 non-null  float64 
 2   longitude   14723 non-null  float64 
 3   geometry    14723 non-null  geometry
 4   HazardType  14723 non-null  int64   
dtypes: float64(2), geometry(1), int64(2)
memory usage: 575.2 KB


In [25]:
ls_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## <font color='red'> 3. Tsunami Data </font>

Data retrieved from NCEI NOAA - Global Historical Tsunami Database

<div class="alert alert-block alert-info">
<b>Note:</b>
    A feature layer displaying historical tsunami events from NCEI's Global Historical Tsunami Database. The Global Historical Tsunami Database consists of two related files containing information on tsunami events from 2000 B.C. to the present in the Atlantic, Indian, and Pacific Oceans; and the Mediterranean and Caribbean Seas.
    
    Citation: National Geophysical Data Center / World Data Service: NCEI/WDS Global Historical Tsunami Database. NOAA National Centers for Environmental Information. doi:10.7289/V5PN93H7 [4 August 2023]

Further info: https://ngdc.noaa.gov/hazard/hazards.shtml

Documentation: https://data.noaa.gov/metaview/page?xml=NOAA/NESDIS/NGDC/MGG/Hazards/iso/xml/G02151.xml&view=getDataView

Layer info: https://www.arcgis.com/home/item.html?id=5a44c3d4d465498993120b70ab568876 
</div>

In [141]:
# Specify the feature class name (replace 'your_feature_class_name' with the actual name)
feature_class_tsunami = "tsunami"  # Change this to your feature class name
tsunami_path = f"{gdb_path}\\{feature_class_tsunami}"

# Describe the feature class
desc_tsun = arcpy.Describe(tsunami_path)

# Check the spatial reference
tsunami_sr = desc_tsun.spatialReference

# Print the spatial reference details
print(f"Spatial Reference of {feature_class_tsunami}:")
print(f"  Name: {tsunami_sr.name}")
print(f"  WKID: {tsunami_sr.factoryCode}")
print(f"  WKT:  {tsunami_sr.exportToString()}")

Spatial Reference of tsunami:
  Name: WGS_1984_Web_Mercator_Auxiliary_Sphere
  WKID: 3857
  WKT:  PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]];-20037700 -30241100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision


In [142]:
# Use arcpy to create a list of fields
ts_fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{feature_class_tsunami}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
ts_data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{feature_class_tsunami}", ts_fields) as cursor:
    for row in cursor:
        ts_data.append(dict(zip(ts_fields, row)))

# Convert the list of dictionaries into a DataFrame
tsun = pd.DataFrame(ts_data)
tsun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   OBJECTID                       2534 non-null   int64  
 1   Shape                          2534 non-null   object 
 2   DEATHS                         252 non-null    float64
 3   DEATHS_DESCRIPTION             346 non-null    object 
 4   CAUSE                          2532 non-null   object 
 5   EVENT_VALIDITY                 2534 non-null   object 
 6   COUNTRY                        2534 non-null   object 
 7   MAX_EVENT_RUNUP                1269 non-null   float64
 8   NUM_RUNUP                      2534 non-null   int64  
 9   YEAR                           2534 non-null   int64  
 10  MONTH                          2422 non-null   float64
 11  DAY                            2339 non-null   float64
 12  HOUR                           1492 non-null   f

In [34]:
ts_df = tsun[['ID','LONGITUDE', 'LATITUDE']].copy()
ts_df.rename(columns = {'ID':'HazardID','LONGITUDE':'longitude', 'LATITUDE':'latitude'}, inplace=True)
ts_df['HazardType'] = 3
ts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HazardID    2534 non-null   int64  
 1   longitude   2534 non-null   float64
 2   latitude    2534 non-null   float64
 3   HazardType  2534 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 79.3 KB


In [35]:
# Generate points geometry from longitude and latitude
ts_df['geometry'] = ts_df.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

# Create a GeoDataFrame
ts_df = gpd.GeoDataFrame(ts_df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
ts_df.set_crs(epsg=4326, inplace=True)

ts_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [36]:
ts_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   HazardID    2534 non-null   int64   
 1   longitude   2534 non-null   float64 
 2   latitude    2534 non-null   float64 
 3   HazardType  2534 non-null   int64   
 4   geometry    2534 non-null   geometry
dtypes: float64(2), geometry(1), int64(2)
memory usage: 99.1 KB


## <font color='red'> 4. Fault Data </font>

GEM Global Active Faults Database (GAF-DB)

<div class="alert alert-block alert-info">
<b>Note:</b>
    A feature layer displaying historical tsunami events from NCEI's Global Historical Tsunami Database. The Global Historical Tsunami Database consists of two related files containing information on tsunami events from 2000 B.C. to the present in the Atlantic, Indian, and Pacific Oceans; and the Mediterranean and Caribbean Seas.
    
    Citation: The GEM GAF-DB has been published in Earthquake Spectra.
    Styron, Richard, and Marco Pagani. “The GEM Global Active Faults Database.” Earthquake Spectra, vol. 36, no. 1_suppl, Oct. 2020, pp. 160–180, doi:10.1177/8755293020944182.

The link to the publication is here: https://journals.sagepub.com/doi/abs/10.1177/8755293020944182

Documentation: https://github.com/GEMScienceTools/gem-global-active-faults

</div>

In [40]:
# Obtain Spatial Reference information for harmonized processing

af_spatial_ref = arcpy.Describe(r"D:/NDIS_Database/05_Fault/fault_points_wgs84.shp").spatialReference
af_spatial_ref

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [21]:
# Read the shapefile from local disk
fault_df = gpd.read_file(r"D:/NDIS_Database/05_Fault/fault_points_wgs84.shp")
fault_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 172823 entries, 0 to 172822
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Id          172823 non-null  int64   
 1   ORIG_FID    172823 non-null  int64   
 2   average_di  79883 non-null   object  
 3   average_ra  51397 non-null   object  
 4   catalog_id  172823 non-null  object  
 5   catalog_na  172823 non-null  object  
 6   dip_dir     35041 non-null   object  
 7   lower_seis  53437 non-null   object  
 8   name        44202 non-null   object  
 9   net_slip_r  110314 non-null  object  
 10  slip_type   169593 non-null  object  
 11  upper_seis  60927 non-null   object  
 12  reference   25748 non-null   object  
 13  epistemic_  72806 non-null   object  
 14  accuracy    19508 non-null   object  
 15  activity_c  69506 non-null   object  
 16  fs_name     20954 non-null   object  
 17  last_movem  8338 non-null    object  
 18  downthrown  1136

In [22]:
fault_df

,Id,ORIG_FID,average_di,average_ra,catalog_id,catalog_na,dip_dir,lower_seis,name,net_slip_r,...,last_movem,downthrown,vert_sep_r,strike_sli,exposure_q,shortening,notes,downthro_1,fault_type,geometry
0,0,0,"(38,,)","(90.0,,)",UCF_2,UCERF3,E,"(16.0,,)",Mount Diablo Thrust,"(1.55,0.8,2.22)",...,None,None,None,None,None,None,None,None,None,POINT (-121.82290 37.73010)
1,0,0,"(38,,)","(90.0,,)",UCF_2,UCERF3,E,"(16.0,,)",Mount Diablo Thrust,"(1.55,0.8,2.22)",...,None,None,None,None,None,None,None,None,None,POINT (-121.86068 37.75582)
2,0,0,"(38,,)","(90.0,,)",UCF_2,UCERF3,E,"(16.0,,)",Mount Diablo Thrust,"(1.55,0.8,2.22)",...,None,None,None,None,None,None,None,None,None,POINT (-121.89846 37.78155)
3,0,0,"(38,,)","(90.0,,)",UCF_2,UCERF3,E,"(16.0,,)",Mount Diablo Thrust,"(1.55,0.8,2.22)",...,None,None,None,None,None,None,None,None,None,POINT (-121.93624 37.80727)
4,0,0,"(38,,)","(90.0,,)",UCF_2,UCERF3,E,"(16.0,,)",Mount Diablo Thrust,"(1.55,0.8,2.22)",...,None,None,None,None,None,None,None,None,None,POINT (-121.97402 37.83299)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172818,0,13695,"(53,40,65)",None,MW_57,SMSSD,NE,None,Bilila-Mtakataka,"(0.13,0.006,0.81)",...,None,None,None,None,None,None,Laterally continuous steep scarp suggests rece...,None,None,POINT (34.57574 -14.43870)
172819,0,13695,"(53,40,65)",None,MW_57,SMSSD,NE,None,Bilila-Mtakataka,"(0.13,0.006,0.81)",...,None,None,None,None,None,None,Laterally continuous steep scarp suggests rece...,None,None,POINT (34.59950 -14.47774)
172820,0,13695,"(53,40,65)",None,MW_57,SMSSD,NE,None,Bilila-Mtakataka,"(0.13,0.006,0.81)",...,None,None,None,None,None,None,Laterally continuous steep scarp suggests rece...,None,None,POINT (34.62327 -14.51678)
172821,0,13695,"(53,40,65)",None,MW_57,SMSSD,NE,None,Bilila-Mtakataka,"(0.13,0.006,0.81)",...,None,None,None,None,None,None,Laterally continuous steep scarp suggests rece...,None,None,POINT (34.64703 -14.55583)


In [42]:
# extract the longitude and latitude from the geometry field using the .x and .y attributes of the geometry column
if fault_df.geometry is not None:
    # Create new fields for longitude and latitude
    fault_df["longitude"] = fault_df.geometry.x
    fault_df["latitude"]  = fault_df.geometry.y

In [74]:
faultdb = fault_df[['catalog_id', 'longitude','latitude', 'geometry']].copy()

In [69]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
faultdb = fault_df[['catalog_id', 'longitude','latitude', 'geometry']].copy()
faultdb.rename(columns = {'catalog_id':'HazardID'}, inplace = True)
faultdb['HazardType'] = 4
faultdb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 172823 entries, 0 to 172822
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   HazardID    172823 non-null  object  
 1   longitude   172823 non-null  float64 
 2   latitude    172823 non-null  float64 
 3   geometry    172823 non-null  geometry
 4   HazardType  172823 non-null  int64   
dtypes: float64(2), geometry(1), int64(1), object(1)
memory usage: 6.6+ MB


## <font color='red'> 5. Earthquake Data </font>

### <font color='green'> Historcal part: </font>

### GHEC Catalog

GEM provides Global Historical Earthquake Catalogue (GHEC) from 1000 to 1903 (Albini, 2014). (https://platform.openquake.org/maps/80/download)

### SHEEC (SHARE European Earthquake Catalogue)

SHEEC catalogue covers the year 1000-1899 for Europe region specifically. It consists of data with magnitude ranges from 1.7 to 8.5 (Stucchi et al., 2012). The data could be downloaded at https: //www.emidius.eu/SHEEC/sheec_1000_1899.html.

### <font color='green'> Instrumental part: </font>

### ISC Bulletin/ISC Global
The ISC Bulletin has now been completely rebuilt for the period 1964-2010. As a result, the ISC hypocentre solutions and magnitudes for the entire period of 1964-latest are based on the ak135 velocity model and the location procedure that is currently used in operations.
(http://www.isc.ac.uk/iscbulletin/search/catalogue/)

The Bulletin of the International Seismological Centre relies on contributions from seismological agencies around the world. To date, a total of 573 agencies have contributed to the ISC Bulletin, throughout its history. For more information about the agency (http://www.isc.ac.uk/iscbulletin/agencies/).


### ISC-GEM Catalogue

The ISC-GEM Global Instrumental Earthquake Catalogue (1904-2016) is the result of a special effort to adapt and substantially extend and improve currently existing bulletin data for large earthquakes (magnitude 5.5 and above, plus continental events down to magnitude 5.0).
(http://www.isc.ac.uk/iscgem/)

### Load data from local layer file

In [51]:
# Fix ID (identifier) of the earthquake events ID
# Load CSV files
df1 = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\eq_1000_2023.csv") # Dataset with incorrect eventID
df2 = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\GHEC1000_1903\eq1000.csv") # Dataset with correct GEHid

# Rename columns in df2 to match df1 (update the mapping if needed)
rename_mapping = {
    "Lat": "latitude",
    "Lon": "longitude",
    "Year": "year",
    "Mo": "month",
    "Da": "day"
}

df2.rename(columns=rename_mapping, inplace=True)

# Define matching fields
key_fields = ["longitude", "latitude", "year", "month", "day"]

# Merge on key fields
merged_df = df1.merge(df2[key_fields + ["GEHid"]], on=key_fields, how="left")

# Replace eventID with GEHid where a match is found
merged_df["eventID"] = merged_df["GEHid"].combine_first(merged_df["eventID"])

# Drop the temporary GEHid column
merged_df.drop(columns=["GEHid"], inplace=True)

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1742244 entries, 0 to 1742243
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   eventID    float64
 1   longitude  float64
 2   latitude   float64
 3   magnitude  float64
 4   year       int64  
 5   month      int64  
 6   day        int64  
 7   hour       int64  
 8   minute     int64  
 9   second     float64
dtypes: float64(5), int64(5)
memory usage: 132.9 MB


In [52]:
merged_df

,eventID,longitude,latitude,magnitude,year,month,day,hour,minute,second
0,210.0,4.2370,50.1830,3.70,1000,3,29,0,0,0.00
1,360.0,38.8000,37.1000,6.20,1003,3,21,0,0,0.00
2,600.0,13.8310,41.4880,5.20,1005,1,1,0,0,0.00
3,500.0,11.8790,43.4630,5.20,1005,1,1,0,0,0.00
4,6849.0,47.4000,34.6000,7.00,1008,4,27,18,0,0.00
...,...,...,...,...,...,...,...,...,...,...
1742239,626591656.0,26.1016,31.0996,4.60,2023,8,3,2,18,10.94
1742240,626591734.0,-117.7490,32.9930,4.89,2023,8,3,7,30,0.00
1742241,626593372.0,70.1037,41.0890,4.00,2023,8,3,10,54,34.32
1742242,626593693.0,93.4950,9.1320,5.28,2023,8,3,11,39,4.00


In [60]:
eq_df = merged_df[['eventID','longitude', 'latitude']].copy()
eq_df.rename(columns = {'eventID':'HazardID'}, inplace = True)
eq_df['HazardType'] = 5
eq_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1742244 entries, 0 to 1742243
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   HazardID    float64 
 1   longitude   float64 
 2   latitude    float64 
 3   HazardType  int64   
 4   geometry    geometry
dtypes: float64(3), geometry(1), int64(1)
memory usage: 66.5 MB


In [56]:
nan_in_eq = eq_df.isnull().sum().sum()
 
# printing the number of values present in
# the whole dataframe
print('Number of NaN values present: ' + str(nan_in_eq))

Number of NaN values present: 0


In [57]:
# Generate points geometry from longitude and latitude
eq_df['geometry'] = eq_df.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

# Create a GeoDataFrame
eq_df = gpd.GeoDataFrame(eq_df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
eq_df.set_crs(epsg=4326, inplace=True)

eq_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [61]:
eq_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1742244 entries, 0 to 1742243
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   HazardID    float64 
 1   longitude   float64 
 2   latitude    float64 
 3   HazardType  int64   
 4   geometry    geometry
dtypes: float64(3), geometry(1), int64(1)
memory usage: 66.5 MB


-------------------------------------------------

--------------------------------------------- Local Layer File Processing Ends Here ----------------------------------------------

-------------------------------------------------

# Concatenate all data

In [63]:
# Remove duplicates in TS based on EQ IDs, keeping EQ IDs
ts_df = ts_df[~ts_df["HazardID"].isin(eq_df["HazardID"])]
ts_df

,HazardID,longitude,latitude,HazardType,geometry
0,1,35.800,35.683,3,POINT (35.80000 35.68300)
1,3,25.400,36.400,3,POINT (25.40000 36.40000)
2,4,35.800,35.683,3,POINT (35.80000 35.68300)
3,5,26.240,39.960,3,POINT (26.24000 39.96000)
4,7,35.220,33.270,3,POINT (35.22000 33.27000)
...,...,...,...,...,...
2529,5965,167.902,-14.464,3,POINT (167.90200 -14.46400)
2530,5966,167.776,-16.895,3,POINT (167.77600 -16.89500)
2531,5969,-82.456,17.702,3,POINT (-82.45600 17.70200)
2532,5970,76.450,-69.417,3,POINT (76.45000 -69.41700)


In [66]:
# Combine with all datasets except for fault
vlte_df = pd.concat([vo_df, ls_df, ts_df, eq_df], ignore_index=True)
vlte_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1760676 entries, 0 to 1760675
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   HazardID    float64 
 1   latitude    float64 
 2   longitude   float64 
 3   geometry    geometry
 4   HazardType  int64   
dtypes: float64(3), geometry(1), int64(1)
memory usage: 67.2 MB


In [70]:
# Generate ID for faults
# First three digits are 777 to identify they are faults data and the rest numbers are randomly generated, unique and not in other datasets
# Load datasets IDs
vo_ids = set(vo_df["HazardID"].astype(int))  # Volcano event IDs
ls_ids = set(ls_df["HazardID"].astype(int))  # Landslide event IDs
ts_ids = set(ts_df["HazardID"].astype(int))  # Tsunami event IDs
eq_ids = set(eq_df["HazardID"].astype(int))  # Earthquake event IDs

# Combine all existing IDs into one set
existing_ids = eq_ids | ts_ids | vo_ids | ls_ids  

# Generate unique fault IDs
num_faults = len(faultdb)  # Number of fault events
fault_ids = set()

while len(fault_ids) < num_faults:
    new_id = int(f"177{np.random.randint(1000000, 9999999)}")  # Format: 777XXXXXXX
    if new_id not in existing_ids:
        fault_ids.add(new_id)

# Assign to fault dataset
faultdb["HazardID"] = list(fault_ids)
faultdb

,HazardID,longitude,latitude,geometry,HazardType
0,1774714883,-121.822900,37.730100,POINT (-121.82290 37.73010),4
1,1772093444,-121.860680,37.755823,POINT (-121.86068 37.75582),4
2,1779957771,-121.898460,37.781546,POINT (-121.89846 37.78155),4
3,1777336331,-121.936240,37.807270,POINT (-121.93624 37.80727),4
4,1773142032,-121.974020,37.832993,POINT (-121.97402 37.83299),4
...,...,...,...,...,...
172818,1776287728,34.575740,-14.438700,POINT (34.57574 -14.43870),4
172819,1771044848,34.599504,-14.477742,POINT (34.59950 -14.47774),4
172820,1779957746,34.623267,-14.516784,POINT (34.62327 -14.51678),4
172821,1778384891,34.647031,-14.555826,POINT (34.64703 -14.55583),4


In [73]:
#Concatenate fault data to the rest of the datasets
ghz_df = pd.concat([vlte_df, faultdb]) #All Geohazards dataframe
ghz_df

,HazardID,latitude,longitude,geometry,HazardType
0,2.100100e+05,50.170000,6.850000,POINT (6.85000 50.17000),1
1,2.100200e+05,45.775000,2.970000,POINT (2.97000 45.77500),1
2,2.100300e+05,42.170000,2.530000,POINT (2.53000 42.17000),1
3,2.100400e+05,38.870000,-4.020000,POINT (-4.02000 38.87000),1
4,2.110040e+05,41.730000,12.700000,POINT (12.70000 41.73000),1
...,...,...,...,...,...
172818,1.776288e+09,-14.438700,34.575740,POINT (34.57574 -14.43870),4
172819,1.771045e+09,-14.477742,34.599504,POINT (34.59950 -14.47774),4
172820,1.779958e+09,-14.516784,34.623267,POINT (34.62327 -14.51678),4
172821,1.778385e+09,-14.555826,34.647031,POINT (34.64703 -14.55583),4


In [74]:
# Save locally in a supported format
output_path = "D:/NDIS_Database/ghz84.gpkg"  # Adjust as needed
ghz_df.to_file(output_path, driver="GPKG")

In [75]:
ghz_df.drop('geometry',axis=1).to_csv("D:/NDIS_Database/ghz84.csv")

## <font color='red'> Countries EEZ Data </font>
Retrieved from Maritime Boundaries and Exclusive Economic Zones (200NM), version 12

https://www.marineregions.org/. https://doi.org/10.14284/632
<div class="alert alert-block alert-info">
<b>Citation:</b>
    Flanders Marine Institute (2024). Union of the ESRI Country shapefile and the Exclusive Economic Zones (version 4). Available online at https://www.marineregions.org/. https://doi.org/10.14284/698. Consulted on 2025-02-20
Further info: 
https://www.marineregions.org/downloads.php#unioneezcountry
</div>

## Test Load

In [5]:
ghz_gpkg = r"D:\NDIS_Database\ghz84.gpkg"
# Load the GeoPackage
ghz = gpd.read_file(ghz_gpkg, layer="ghz84")
ghz

,HazardID,latitude,longitude,HazardType,geometry
0,2.100100e+05,50.170000,6.850000,1,POINT (6.85000 50.17000)
1,2.100200e+05,45.775000,2.970000,1,POINT (2.97000 45.77500)
2,2.100300e+05,42.170000,2.530000,1,POINT (2.53000 42.17000)
3,2.100400e+05,38.870000,-4.020000,1,POINT (-4.02000 38.87000)
4,2.110040e+05,41.730000,12.700000,1,POINT (12.70000 41.73000)
...,...,...,...,...,...
1933494,1.776288e+09,-14.438700,34.575740,4,POINT (34.57574 -14.43870)
1933495,1.771045e+09,-14.477742,34.599504,4,POINT (34.59950 -14.47774)
1933496,1.779958e+09,-14.516784,34.623267,4,POINT (34.62327 -14.51678)
1933497,1.778385e+09,-14.555826,34.647031,4,POINT (34.64703 -14.55583)


In [6]:
ghz.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

----------------------------------------------------------------------------------

# <font color='red'> Clip and Near Analysis </font>

### CLIP and NEAR Analysis for the entire region

Divide the dataset into regions EEZ (country+ocean territory)

In [11]:
# Read the shapefile from local disk
eez = gpd.read_file(r"D:\NDIS_Database\00_ClippingRegion\EEZ_land_union_v4_202410\EEZ_land_union_v4_202410.shp")
eez.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   MRGID_EEZ   286 non-null    float64 
 1   TERRITORY1  327 non-null    object  
 2   MRGID_TER1  326 non-null    float64 
 3   ISO_TER1    293 non-null    object  
 4   UN_TER1     289 non-null    float64 
 5   SOVEREIGN1  327 non-null    object  
 6   MRGID_SOV1  327 non-null    float64 
 7   ISO_SOV1    327 non-null    object  
 8   POL_TYPE    328 non-null    object  
 9   Y_1         328 non-null    float64 
 10  x_1         328 non-null    float64 
 11  AREA_KM2    328 non-null    int64   
 12  geometry    328 non-null    geometry
dtypes: float64(6), geometry(1), int64(1), object(5)
memory usage: 33.4+ KB


---

### EEZ Data Treatment

In [6]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # Update the path accordingly
gdb_path

'D:\\ArcGISProjects\\GeohazardDB\\GeohazardDB.gdb'

---

In [13]:
# Define feature classes
eez_country = "EEZ_country"
eez_union = "EEZ_union"

# Step 1: Generate Near Table
near_table = "EEZ_near_table"
if not arcpy.Exists(near_table):
    print("⚠️ Generating Near Table...")
    arcpy.GenerateNearTable_analysis(eez_union, eez_country, near_table, closest="CLOSEST", method="PLANAR")

# Step 2: Add NEAR_FID to EEZ_union
arcpy.JoinField_management(eez_union, "OBJECTID", near_table, "IN_FID", ["NEAR_FID"])

# Step 3: Merge EEZ_union into EEZ_country using NEAR_FID
with arcpy.da.UpdateCursor(eez_country, ["OBJECTID", "SHAPE@"]) as country_cursor:
    for country_row in country_cursor:
        country_id = country_row[0]
        country_geom = country_row[1]

        # Collect matching EEZ_union geometries
        union_geoms = []
        with arcpy.da.SearchCursor(eez_union, ["NEAR_FID", "SHAPE@"]) as union_cursor:
            for union_row in union_cursor:
                if union_row[0] == country_id and union_row[1] is not None:
                    union_geoms.append(union_row[1])

        # Merge all geometries if matches exist
        if union_geoms:
            merged_geom = reduce(lambda x, y: x.union(y), [country_geom] + union_geoms)
            country_row[1] = merged_geom
            country_cursor.updateRow(country_row)

print("✅ Finished merging polygons. EEZ_country now contains 275 polygons.")

✅ Finished merging polygons. EEZ_country now contains 275 polygons.


In [18]:
# Define feature classes
eez = "EEZ24"
# Field to check (change if needed)
unique_field = "FID"

# Get unique values
unique_values = set()
with arcpy.da.SearchCursor(eez, [unique_field]) as cursor:
    for row in cursor:
        if row[0]:  # Ignore Null values
            unique_values.add(row[0])

# Print results
print(f"✅ Unique {unique_field} count: {len(unique_values)}")
if len(unique_values) == 327:
    print("🎉 The field has exactly 327 unique values, matching the number of polygons!")
else:
    print(f"⚠️ Mismatch! Expected 327 but found {len(unique_values)}. Check for duplicates or missing values.")

✅ Unique FID count: 327
🎉 The field has exactly 327 unique values, matching the number of polygons!


In [16]:
# Field to check
field_name = "ISO_SOV1"
search_value = "AAT"

# Check if "ANT" exists
exists = False
with arcpy.da.SearchCursor(eez, [field_name]) as cursor:
    for row in cursor:
        if row[0] == search_value:
            exists = True
            break  # No need to continue once found

# Print result
if exists:
    print(f"✅ '{search_value}' exists in the field '{field_name}'.")
else:
    print(f"❌ '{search_value}' was NOT found in '{field_name}'.")

❌ 'AAT' was NOT found in 'ISO_SOV1'.


In [12]:
import random
import string

In [4]:
input_eez = "EEZ24"

# Create a set to track used ISO_TER1 values and if NaN or duplicates, rename it from TERRITORY1 abbreviation
used_names = set()

# Function to generate a unique 3-letter code
def generate_unique_code(existing_names):
    while True:
        # Generate a random 3-letter combination
        code = ''.join(random.choices(string.ascii_uppercase, k=3))
        if code not in existing_names:  # Ensure it's unique
            return code

# Step 1: Update the ISO_TER1 values
with arcpy.da.UpdateCursor(input_eez, ['ISO_TER1', 'TERRITORY1']) as cursor:
    for row in cursor:
        iso_name = row[0]
        territory_name = row[1]

        # Check if this ISO_TER1 value is empty or not valid
        if not iso_name or len(iso_name) != 3 or not iso_name.isalpha():
            # Generate a unique 3-letter code
            unique_code = generate_unique_code(used_names)
            row[0] = unique_code  # Update ISO_TER1 value
            used_names.add(unique_code)  # Add to the set of used names

        # If it is valid and unique, keep the original value
        cursor.updateRow(row)

print("✅ ISO_TER1 values have been updated to unique 3-letter names.")

✅ ISO_TER1 values have been updated to unique 3-letter names.


In [8]:
from collections import defaultdict

# Set input dataset
input_eez = "EEZ"

# Track used ISO_TER1 values
used_names = set()

# Track duplicates: {ISO_TER1: [rows]}
iso_to_rows = defaultdict(list)

# Step 1: First Pass - Identify duplicates and invalid values
with arcpy.da.UpdateCursor(input_eez, ['ISO_TER1', 'UNION']) as cursor:
    for row in cursor:
        iso_name = row[0]
        territory_name = row[1]

        # Store existing values for duplication checks
        if iso_name and len(iso_name) == 3 and iso_name.isalpha():
            if iso_name in used_names:
                iso_to_rows[iso_name].append(row)  # Mark as duplicate
            else:
                used_names.add(iso_name)  # Add valid unique code
        else:
            iso_to_rows["INVALID"].append(row)  # Mark invalid values

# Function to generate a unique 3-letter code
def generate_unique_code(prefix, existing_names):
    while True:
        # Generate a code using the first 2 letters of UNION + 1 random letter
        if prefix and len(prefix) >= 2:
            code = prefix[:2].upper() + random.choice(string.ascii_uppercase)
        else:
            code = ''.join(random.choices(string.ascii_uppercase, k=3))

        if code not in existing_names:  # Ensure uniqueness
            return code

# Step 2: Second Pass - Fix duplicates and invalid values
with arcpy.da.UpdateCursor(input_eez, ['ISO_TER1', 'UNION']) as cursor:
    for row in cursor:
        iso_name = row[0]
        territory_name = row[1]

        # If this row was marked as a duplicate or invalid, update it
        if iso_name in iso_to_rows or iso_name == "INVALID":
            prefix = territory_name[:2] if territory_name else ""
            unique_code = generate_unique_code(prefix, used_names)
            row[0] = unique_code
            used_names.add(unique_code)

        cursor.updateRow(row)

print("✅ ISO_TER1 values updated: duplicates fixed, invalid values replaced.")

✅ ISO_TER1 values updated: duplicates fixed, invalid values replaced.


---------

### Test Using Smaller area

-----

In [14]:
start_time = timeit.default_timer()
# Define paths
project_folder = r"D:\ArcGISProjects\GeohazardDB"
gdb_path = os.path.join(project_folder, "GeohazardDB.gdb")  # File Geodatabase

# Initialize a list to store processed geohazard datasets
ghz_list = []

# Input datasets
geohazard_layer = os.path.join(gdb_path, "g84_Clip")  # Geohazard dataset
road_layer = os.path.join(gdb_path, "grip_Clip")  # Global road dataset
country_layer = os.path.join(gdb_path, "eez3")  # Country boundaries

# Check if GDB exists, otherwise create it
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(project_folder, "GeohazardDB.gdb")

# Iterate through each country
with arcpy.da.SearchCursor(country_layer, ["ISO_TER1", "SHAPE@"]) as country_cursor:
    for row in country_cursor:
        country_code = row[0]  # Country code (e.g., THA, USA)
        country_geometry = row[1]  # Country boundary geometry

        print(f"\u23F3 Processing country: {country_code}...")

        # Define output names inside the GDB
        ghz_clip = os.path.join(gdb_path, f"ghz_{country_code}")
        road_clip = os.path.join(gdb_path, f"road_{country_code}")
        near_output = os.path.join(gdb_path, f"near_{country_code}")

        # ---- Step 1: Clip Geohazard Data ----
        if arcpy.Exists(ghz_clip):
            arcpy.Delete_management(ghz_clip)
        try:
            arcpy.Clip_analysis(geohazard_layer, country_geometry, ghz_clip)
            print(f"  \u2705 Clipped geohazard layer: {ghz_clip}")
        except Exception as e:
            print(f"  \u274C Error clipping geohazard: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 2: Clip Road Data ----
        if arcpy.Exists(road_clip):
            arcpy.Delete_management(road_clip)
        try:
            arcpy.Clip_analysis(road_layer, country_geometry, road_clip)
            print(f"  \u2705 Clipped road layer: {road_clip}")
        except Exception as e:
            print(f"  {chr(0x274C)} Error clipping roads: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 3: Perform Near Analysis ----
        if arcpy.Exists(near_output):
            arcpy.Delete_management(near_output)
        try:
            arcpy.GenerateNearTable_analysis(
                in_features=ghz_clip,
                near_features=road_clip,
                out_table=near_output,
                search_radius="100000 Meters",  # Keep it in 100 Km
                location="LOCATION",  # Include X, Y coordinates
                angle="ANGLE",
                closest="CLOSEST",
                method="GEODESIC"
            )
            print(f"  \u2705 Near analysis completed: {near_output}")

        except Exception as e:
            print(f"  \u274C Error in Near Analysis: {e}")


        # ---- Step 4: Add distance to geohazard ----
        # Add "distance" field if it doesn't exist
        if "distance" not in [f.name for f in arcpy.ListFields(ghz_clip)]:
            arcpy.AddField_management(ghz_clip, "distance", "DOUBLE")

        # Update "distance" field with NEAR_DIST from the near table
        with arcpy.da.UpdateCursor(ghz_clip, ["OBJECTID", "distance"]) as ghz_cursor:
            for ghz_row in ghz_cursor:
                ghz_id = ghz_row[0]  # OBJECTID of ghz_XXX
                
                # Find matching NEAR_DIST from the near table
                with arcpy.da.SearchCursor(near_output, ["IN_FID", "NEAR_DIST"]) as near_cursor:
                    for near_row in near_cursor:
                        if near_row[0] == ghz_id:  # Match OBJECTID
                            ghz_row[1] = near_row[1]  # Assign NEAR_DIST
                            ghz_cursor.updateRow(ghz_row)
                            break  # Stop once found
            
        print(f"\u2705 NEAR_DIST added to {ghz_clip} as 'distance' field.")
        
        # Add "HazardID" field to near table if it doesn't exist
        if "HazardID" not in [f.name for f in arcpy.ListFields(near_output)]:
            arcpy.AddField_management(near_output, "HazardID", "TEXT")
        
        # Since this uses "CLOSEST" method, so it will only take 1 closest disance,
        # and discard the NEAR_FID as the 1:N will no longer needed. Instead it can be replaced with HazardID
        # to obtain the relationship with the ghz dataset
        # Use UpdateCursor to populate HazardID from ghz_XXX
        with arcpy.da.UpdateCursor(near_output, ["IN_FID", "HazardID"]) as cursor:
            for row in cursor:
                # Fetch the corresponding HazardID from ghz_XXX
                with arcpy.da.SearchCursor(ghz_clip, ["OBJECTID", "HazardID"]) as ghz_cursor:
                    for ghz_row in ghz_cursor:
                        if row[0] == ghz_row[0]:  # Match OBJECTID
                            row[1] = ghz_row[1]  # Assign HazardID
                            cursor.updateRow(row)
                            break  # Exit loop once matched

        print(f"\u2705 HazardID added to {near_output}, replacing NEAR_FID reference.")

        # Add processed dataset to list for final merge
        ghz_list.append(ghz_clip)

# Merge all processed ghz_XXX datasets into one: "ghz_dist"
ghz_dist = os.path.join(gdb_path, "ghz_dist")

if arcpy.Exists(ghz_dist):
    arcpy.Delete_management(ghz_dist)  # Ensure a fresh start

arcpy.Merge_management(ghz_list, ghz_dist)

print(f" All geohazard data merged into {ghz_dist} successfully!")

elapsed = timeit.default_timer() - start_time
print("\u2705 All processing completed! Elapsed time: %s minutes"%str(elapsed/60))

⏳ Processing country: GRD...
  ✅ Clipped geohazard layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_GRD
  ✅ Clipped road layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\road_GRD
  ✅ Near analysis completed: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_GRD
✅ NEAR_DIST added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_GRD as 'distance' field.
✅ HazardID added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_GRD, replacing NEAR_FID reference.
⏳ Processing country: MAF...
  ✅ Clipped geohazard layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_MAF
  ✅ Clipped road layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\road_MAF
  ✅ Near analysis completed: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_MAF
✅ NEAR_DIST added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_MAF as 'distance' field.
✅ HazardID added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_MAF, replacing NEAR_FID reference.
⏳ Processing country: MTQ...
  ✅ Clipped geoha

-----

# <h1 style="background-color:#d5f2e1; color: #bc6ee0;">Clip and Near Analysis Including Outside EEZ</h1>

---

In [10]:
start_time = timeit.default_timer()

# Define paths
project_folder = r"D:\ArcGISProjects\GeohazardDB"
gdb_path = os.path.join(project_folder, "GeohazardDB.gdb")  # File Geodatabase

# Initialize lists to store processed geohazard datasets and near tables
ghz_list = []
near_tables = []
outside_eez_list = []  # List to store datasets with points outside EEZ

# Input datasets
geohazard_layer = os.path.join(gdb_path, "ghz84")  # Geohazard dataset
road_layer = os.path.join(gdb_path, "roads")  # Global road dataset
country_layer = os.path.join(gdb_path, "eez_country")  # Country boundaries

# Check if GDB exists, otherwise create it
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(project_folder, "GeohazardDB.gdb")

# Get total number of countries to process
total_countries = int(arcpy.GetCount_management(country_layer)[0])

# Iterate through each country
with arcpy.da.SearchCursor(country_layer, ["ISO_TER1", "SHAPE@"]) as country_cursor:
    for index, row in enumerate(country_cursor, start=1):
        country_code = row[0]  # Country code (e.g., THA, USA)
        country_geometry = row[1]  # Country boundary geometry

        print(f"\u23F3 Processing country {index}/{total_countries}: {country_code}...")

        # Define output names inside the GDB
        ghz_clip = os.path.join(gdb_path, f"ghz_{country_code}")
        road_clip = os.path.join(gdb_path, f"road_{country_code}")
        near_output = os.path.join(gdb_path, f"near_{country_code}")

        # ---- Step 1: Clip Geohazard Data ----
        if arcpy.Exists(ghz_clip):
            arcpy.Delete_management(ghz_clip)
        try:
            arcpy.Clip_analysis(geohazard_layer, country_geometry, ghz_clip)
            print(f"  \u2705 Clipped geohazard layer: {ghz_clip}")
        except Exception as e:
            print(f"  \u274C Error clipping geohazard: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 2: Clip Road Data ----
        if arcpy.Exists(road_clip):
            arcpy.Delete_management(road_clip)
        try:
            arcpy.Clip_analysis(road_layer, country_geometry, road_clip)
            print(f"  \u2705 Clipped road layer: {road_clip}")
        except Exception as e:
            print(f"  {chr(0x274C)} Error clipping roads: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 3: Perform Near Analysis ----
        if arcpy.Exists(near_output):
            arcpy.Delete_management(near_output)
        try:
            arcpy.GenerateNearTable_analysis(
                in_features=ghz_clip,
                near_features=road_clip,
                out_table=near_output,
                search_radius="",  # No search radius
                location="LOCATION",  # Include X, Y coordinates
                angle="ANGLE",
                closest="CLOSEST",
                method="GEODESIC"
            )
            print(f"  \u2705 Near analysis completed: {near_output}")
            near_tables.append(near_output)  # Store near table

        except Exception as e:
            print(f"  \u274C Error in Near Analysis: {e}")

        # ---- Step 4: Add distance to geohazard ----
        # Add "distance" field if it doesn't exist
        if "distance" not in [f.name for f in arcpy.ListFields(ghz_clip)]:
            arcpy.AddField_management(ghz_clip, "distance", "DOUBLE")

        # Update "distance" field with NEAR_DIST from the near table
        with arcpy.da.UpdateCursor(ghz_clip, ["OBJECTID", "distance"]) as ghz_cursor:
            for ghz_row in ghz_cursor:
                ghz_id = ghz_row[0]  # OBJECTID of ghz_XXX
                
                # Find matching NEAR_DIST from the near table
                found_match = False
                with arcpy.da.SearchCursor(near_output, ["IN_FID", "NEAR_DIST"]) as near_cursor:
                    for near_row in near_cursor:
                        if near_row[0] == ghz_id:  # Match OBJECTID
                            ghz_row[1] = near_row[1]  # Assign NEAR_DIST
                            ghz_cursor.updateRow(ghz_row)
                            found_match = True
                            break  # Stop once found
                
                if not found_match:  # If no match found, add to outside_eez list
                    outside_eez_list.append(ghz_clip)

        print(f"\u2705 NEAR_DIST added to {ghz_clip} as 'distance' field.")
        
        # Add "HazardID" field to near table if it doesn't exist
        if "HazardID" not in [f.name for f in arcpy.ListFields(near_output)]:
            arcpy.AddField_management(near_output, "HazardID", "TEXT")
        
        # Use UpdateCursor to populate HazardID from ghz_XXX
        with arcpy.da.UpdateCursor(near_output, ["IN_FID", "HazardID"]) as cursor:
            for row in cursor:
                # Fetch the corresponding HazardID from ghz_XXX
                with arcpy.da.SearchCursor(ghz_clip, ["OBJECTID", "HazardID"]) as ghz_cursor:
                    for ghz_row in ghz_cursor:
                        if row[0] == ghz_row[0]:  # Match OBJECTID
                            row[1] = ghz_row[1]  # Assign HazardID
                            cursor.updateRow(row)
                            break  # Exit loop once matched

        print(f"\u2705 HazardID added to {near_output}, replacing NEAR_FID reference.")

        # Add processed dataset to list for final merge
        ghz_list.append(ghz_clip)

# Merge all processed ghz_XXX datasets into one: "ghz_dist"
ghz_dist = os.path.join(gdb_path, "ghz_dist")

if arcpy.Exists(ghz_dist):
    arcpy.Delete_management(ghz_dist)  # Ensure a fresh start

arcpy.Merge_management(ghz_list, ghz_dist)

print(f" All geohazard data merged into {ghz_dist} successfully!")

# ---- Step 5: Compile all near tables into one ----
compiled_near_table = os.path.join(gdb_path, "compiled_near_table")
if arcpy.Exists(compiled_near_table):
    arcpy.Delete_management(compiled_near_table)

# Create empty table to store results
arcpy.CreateTable_management(gdb_path, "compiled_near_table")
arcpy.AddField_management(compiled_near_table, "FROM_X", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "FROM_Y", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "NEAR_X", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "NEAR_Y", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "NEAR_FID", "LONG")
arcpy.AddField_management(compiled_near_table, "HazardID", "TEXT")

# Insert cursor for compiled near table
with arcpy.da.InsertCursor(compiled_near_table, ["FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y", "NEAR_FID", "HazardID"]) as insert_cursor:
    for near_table in near_tables:
        with arcpy.da.SearchCursor(near_table, ["FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y", "NEAR_FID", "HazardID"]) as cursor:
            for row in cursor:
                insert_cursor.insertRow(row)

print(f"\u270 All near tables compiled into {compiled_near_table} successfully!")

# ---- Step 6: Handle points outside EEZ ----
outside_eez_output = os.path.join(gdb_path, "outside_eez")

# Try deleting the feature class if it exists (avoid locking issues)
if arcpy.Exists(outside_eez_output):
    try:
        arcpy.Delete_management(outside_eez_output)
        print(f"\u270 Deleted existing {outside_eez_output}")
    except Exception as e:
        print(f"&#9888 Error deleting {outside_eez_output}: {e}")
        time.sleep(2)  # Wait a bit and retry
        arcpy.Delete_management(outside_eez_output)

# Get all field names from geohazard_layer (excluding OBJECTID & Shape)
fields = [f.name for f in arcpy.ListFields(geohazard_layer) if f.type not in ("OID", "Geometry")]
fields.insert(0, "SHAPE@")  # Ensure geometry is included

# Create a new feature class with the same schema
try:
    arcpy.CreateFeatureclass_management(
        gdb_path, "outside_eez", "POINT",
        template=geohazard_layer,  # Preserve schema
        spatial_reference=arcpy.Describe(geohazard_layer).spatialReference
    )
    print("\u270 Successfully created outside_eez feature class")
except Exception as e:
    print(f"\u274C Failed to create outside_eez: {e}")

# ---- Step 7: Create Polyline Feature Class from Compiled Near Table ----
polyline_output = os.path.join(gdb_path, "compiled_near_lines")
if arcpy.Exists(polyline_output):
    arcpy.Delete_management(polyline_output)

# Use XY To Line tool to create lines from compiled near table
arcpy.XYToLine_management(
    compiled_near_table,
    polyline_output,
    "FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y",
    "",  # Optional Line ID field
)

print(f"\u270 Polylines created from compiled near table at {polyline_output}!")


elapsed = timeit.default_timer() - start_time
print("\u2705 All processing completed! Elapsed time: %s minutes" % str(elapsed / 60))

⏳ Processing country 1/328: JOR...
  ✅ Clipped geohazard layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_JOR
  ✅ Clipped road layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\road_JOR
  ✅ Near analysis completed: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_JOR
✅ NEAR_DIST added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_JOR as 'distance' field.
✅ HazardID added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_JOR, replacing NEAR_FID reference.
⏳ Processing country 2/328: BDI...
  ✅ Clipped geohazard layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_BDI
  ✅ Clipped road layer: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\road_BDI
  ✅ Near analysis completed: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_BDI
✅ NEAR_DIST added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_BDI as 'distance' field.
✅ HazardID added to D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\near_BDI, replacing NEAR_FID reference.
⏳ Processing country 3/328: URJ...

In [11]:
4628.1864107050005/60

77.13644017841668

In [ ]:
# Paths
source_gdb = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"
output_gdb = r"D:\ArcGISProjects\GeohazardDB\NDIS.gdb"

# Make sure output GDB exists
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

# Example test country codes — replace or expand this list as needed
test_country_codes = ["THA", "IDN", "JPN"]

for country_code in test_country_codes:
    ghz_fc = os.path.join(source_gdb, f"ghz_{country_code}")
    road_fc = os.path.join(source_gdb, f"road_{country_code}")
    near_table = os.path.join(output_gdb, f"near_{country_code}")
    point_output = os.path.join(output_gdb, f"near_points_{country_code}")

    print(f"\n--- Processing {country_code} ---")
    
    # Check if input features exist
    if not arcpy.Exists(ghz_fc) or not arcpy.Exists(road_fc):
        print(f"  ⛔ Skipped: Missing clipped layers for {country_code}")
        continue

    try:
        # Step 1: Generate Near Table
        if arcpy.Exists(near_table):
            arcpy.Delete_management(near_table)
        arcpy.GenerateNearTable_analysis(
            in_features=ghz_fc,
            near_features=road_fc,
            out_table=near_table,
            location="LOCATION",
            angle="ANGLE",
            closest="CLOSEST",
            method="GEODESIC"
        )
        print(f"  ✅ Near table created: {near_table}")
    except Exception as e:
        print(f"  ❌ Error generating near table: {e}")
        continue

    try:
        # Step 2: Convert XY to Point
        if arcpy.Exists(point_output):
            arcpy.Delete_management(point_output)
        arcpy.XYTableToPoint_management(
            in_table=near_table,
            out_feature_class=point_output,
            x_field="NEAR_X",
            y_field="NEAR_Y",
            coordinate_system=arcpy.Describe(ghz_fc).spatialReference
        )
        print(f"  ✅ XY to Point completed: {point_output}")
    except Exception as e:
        print(f"  ❌ Error converting to point: {e}")


--- Processing THA ---
  ✅ Near table created: D:\ArcGISProjects\GeohazardDB\NDIS.gdb\near_THA
  ✅ XY to Point completed: D:\ArcGISProjects\GeohazardDB\NDIS.gdb\near_points_THA

--- Processing IDN ---
  ✅ Near table created: D:\ArcGISProjects\GeohazardDB\NDIS.gdb\near_IDN
  ✅ XY to Point completed: D:\ArcGISProjects\GeohazardDB\NDIS.gdb\near_points_IDN

--- Processing JPN ---


In [28]:
# ---- Step 6: Handle points outside EEZ ----
outside_eez_output = os.path.join(gdb_path, "outside_eez")

# Try deleting the feature class if it exists (avoid locking issues)
if arcpy.Exists(outside_eez_output):
    try:
        arcpy.Delete_management(outside_eez_output)
        print(f"✅ Deleted existing {outside_eez_output}")
    except Exception as e:
        print(f"⚠️ Error deleting {outside_eez_output}: {e}")
        time.sleep(2)  # Wait a bit and retry
        arcpy.Delete_management(outside_eez_output)

# Get all field names from geohazard_layer (excluding OBJECTID & Shape)
fields = [f.name for f in arcpy.ListFields(geohazard_layer) if f.type not in ("OID", "Geometry")]
fields.insert(0, "SHAPE@")  # Ensure geometry is included

# Create a new feature class with the same schema
try:
    arcpy.CreateFeatureclass_management(
        gdb_path, "outside_eez", "POINT",
        template=geohazard_layer,  # Preserve schema
        spatial_reference=arcpy.Describe(geohazard_layer).spatialReference
    )
    print("✅ Successfully created outside_eez feature class")
except Exception as e:
    print(f"❌ Failed to create outside_eez: {e}")

✅ Deleted existing D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\outside_eez
✅ Successfully created outside_eez feature class


In [29]:
# ---- Step 7: Create Polyline Feature Class from Compiled Near Table ----
polyline_output = os.path.join(gdb_path, "compiled_near_lines")
if arcpy.Exists(polyline_output):
    arcpy.Delete_management(polyline_output)

# Use XY To Line tool to create lines from compiled near table
arcpy.XYToLine_management(
    compiled_near_table,
    polyline_output,
    "FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y",
    "",  # Optional Line ID field
)

print(f"✅ Polylines created from compiled near table at {polyline_output}!")

✅ Polylines created from compiled near table at D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\compiled_near_lines!


----

# Statistics of Database

In [3]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # This gdb path

In [12]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # This gdb path
# Specify the feature class name (replace 'your_feature_class_name' with the actual name)
ghz_dist = "ghz_dist"  # Geohazard feature class
ghz_path = f"{gdb_path}\\{ghz_dist}"

# Use arcpy to create a list of fields
ghz_fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{ghz_dist}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
ghz_data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{ghz_dist}", ghz_fields) as cursor:
    for row in cursor:
        ghz_data.append(dict(zip(ghz_fields, row)))

# Convert the list of dictionaries into a DataFrame
ghzdf = pd.DataFrame(ghz_data)
ghzdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1855013 entries, 0 to 1855012
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   OBJECTID    int64  
 1   Shape       object 
 2   HazardID    float64
 3   latitude    float64
 4   longitude   float64
 5   HazardType  int64  
 6   distance    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 99.1+ MB


In [4]:
nan_in_ghz = ghzdf.isnull().sum().sum()
 
# printing the number of values present in
# the whole dataframe
print('Number of NaN values present: ' + str(nan_in_ghz))

Number of NaN values present: 36489


In [14]:
1845470-1855013

-9543

### Check layer input

In [6]:
# Input datasets
geohazard_layer = os.path.join(gdb_path, "ghz84")  # Geohazard dataset
road_layer = os.path.join(gdb_path, "roads")  # Global road dataset
country_layer = os.path.join(gdb_path, "eez_country")  # Country boundaries

In [7]:
print("Geohazard CRS:", arcpy.Describe(geohazard_layer).spatialReference.name)
print("Road CRS:", arcpy.Describe(road_layer).spatialReference.name)
print("Countries CRS:", arcpy.Describe(country_layer).spatialReference.name)

Geohazard CRS: GCS_WGS_1984
Road CRS: GCS_WGS_1984
Countries CRS: GCS_WGS_1984


In [8]:
count_ghz = arcpy.GetCount_management(geohazard_layer)
count_road = arcpy.GetCount_management(road_layer)
print(f"Geohazard points: {count_ghz}")
print(f"Road features: {count_road}")

Geohazard points: 1933499
Road features: 25758453


In [21]:
# Field to check for null values
dist_name = "Shape"

# Initialize counter for null values
null_count = 0

# Use SearchCursor to iterate through the feature class
with arcpy.da.SearchCursor(geohazard_layer, [dist_name]) as cursor:
    for row in cursor:
        # Check if the field is null (None)
        if row[0] is None:
            null_count += 1

# Output the number of null values
print(f"Number of null values in '{dist_name}' field: {null_count}")

Number of null values in 'Shape' field: 0


### Separate null data and remove duplicates

In [9]:
# Specify the feature class name (replace 'your_feature_class_name' with the actual name)
ghz84 = "ghz84"  # Geohazard feature class
ghz84_path = f"{gdb_path}\\{ghz84}"

# Use arcpy to create a list of fields
ghz84_fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{ghz84}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
ghz84_data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{ghz84}", ghz84_fields) as cursor:
    for row in cursor:
        ghz84_data.append(dict(zip(ghz84_fields, row)))

# Convert the list of dictionaries into a DataFrame
ghz84df = pd.DataFrame(ghz84_data)
ghz84df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933499 entries, 0 to 1933498
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   OBJECTID    int64  
 1   Shape       object 
 2   HazardID    float64
 3   latitude    float64
 4   longitude   float64
 5   HazardType  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 88.5+ MB


In [11]:
# Separate rows with null (NaN) distance
df_no_nulls = ghzdf[ghzdf['distance'].notna()]

# Separate rows with null distance (NaN)
df_null_distance = ghzdf[ghzdf['distance'].isna()]

# Now, handle duplicates by HazardID - keep the row with the smallest distance
df_no_nulls = df_no_nulls.loc[df_no_nulls.groupby('HazardID')['distance'].idxmin()]

<class 'NameError'>: name 'ghzdf' is not defined

In [16]:
# Print results
print(f"✅ Total input points: {len(ghz84df)}, non null: {len(df_no_nulls)}, null: {len(df_null_distance)}. Output: {len(df_no_nulls)+len(df_null_distance)}. \n Difference from original input: {len(ghz84df)-len(df_no_nulls)}")

✅ Total input points: 1933499, non null: 1814699, null: 36489. Output: 1851188. 
 Difference from original input: 118800


In [6]:
geohazard_layer = os.path.join(gdb_path, "ghz_dist")

# Define the output feature class
output_fc = os.path.join(gdb_path, "cleaned_geohazard_data")

# Retrieve data from feature class and store it in DataFrame
fields = ["OBJECTID", "Shape@", "HazardID", "latitude", "longitude", "HazardType", "distance"]
data = []

# Use SearchCursor to extract data from feature class
with arcpy.da.SearchCursor(geohazard_layer, fields) as cursor:
    for row in cursor:
        # Separate geometry from the non-geometry fields
        geometry = row[1]  # Shape@ is at index 1
        data.append(row[:1] + (geometry,) + row[2:])  # Append geometry separately

# Create DataFrame without geometry
df = pd.DataFrame(data, columns=["OBJECTID", "Shape@", "HazardID", "latitude", "longitude", "HazardType", "distance"])

# Convert 'distance' field to numeric, handling errors as NaN
df['distance'] = pd.to_numeric(df['distance'], errors='coerce')

# Filter out rows with NaN distance values
df_no_nulls = df[df['distance'].notna()]

# Handle duplicates by HazardID (keep row with smallest distance)
df_no_nulls = df_no_nulls.loc[df_no_nulls.groupby('HazardID')['distance'].idxmin()]

# Create the output feature class with the correct schema (keeping the geometry)
# Create the new feature class based on the template (you can use the original one to copy schema)
arcpy.management.CreateFeatureclass(
    out_path=gdb_path,
    out_name="cleaned_geohazard_data",
    geometry_type="POINT",  # Change to 'POINT' if the geometry is a point
    template=geohazard_layer,  # Copy schema from the original feature class
    spatial_reference=arcpy.Describe(geohazard_layer).spatialReference
)

# Insert cleaned data into the new feature class using InsertCursor
with arcpy.da.InsertCursor(output_fc, ["SHAPE@", "HazardID", "latitude", "longitude", "HazardType", "distance"]) as cursor:
    for idx, row in df_no_nulls.iterrows():
        geometry = row['Shape@']  # Retrieve geometry separately
        cursor.insertRow([geometry, row['HazardID'], row['latitude'], row['longitude'], row['HazardType'], row['distance']])

print(f"Cleaned data written to: {output_fc}")
# Play sound to notify that it's done
# playsound(r"C:\Windows\Media\Alarm09.wav")

Cleaned data written to: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\cleaned_geohazard_data


In [12]:
# Specify the feature class name (replace 'your_feature_class_name' with the actual name)
ghz_clean = "cleaned_geohazard_data"  # Geohazard feature class
ghz_clean_path = f"{gdb_path}\\{ghz_clean}"

# Use arcpy to create a list of fields
ghz_clean_fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{ghz_clean}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
ghz_clean_data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{ghz_clean}", ghz_clean_fields) as cursor:
    for row in cursor:
        ghz_clean_data.append(dict(zip(ghz_clean_fields, row)))

# Convert the list of dictionaries into a DataFrame
ghz_celan_df = pd.DataFrame(ghz_clean_data)
ghz_celan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1814699 entries, 0 to 1814698
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   OBJECTID    int64  
 1   Shape       object 
 2   HazardID    float64
 3   latitude    float64
 4   longitude   float64
 5   HazardType  int64  
 6   distance    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 96.9+ MB


In [14]:
# Print results
print(f"✅ Total input points: {len(ghz84df)}, cleaned: {len(ghz_celan_df)}. Difference: {len(ghz84df)-len(ghz_celan_df)}.")

✅ Total input points: 1933499, cleaned: 1814699. Difference: 118800.


In [20]:
# Find HazardIDs in the raw input dataset that are NOT in the processed dataset
raw_input_ids = set(ghz84df['HazardID'])
processed_ids = set(ghz_celan_df['HazardID'])

# HazardIDs that are in the raw input but not in processed
unprocessed_ids = raw_input_ids - processed_ids

# Filter the raw input DataFrame to get the rows that are not processed (i.e., not in the processed dataset)
unprocessed_data = ghz84df[ghz84df['HazardID'].isin(unprocessed_ids)]

print("Unprocessed Data:")
print(unprocessed_data)

Unprocessed Data:
         OBJECTID                                     Shape      HazardID  \
182           183  (77.52000000000004, -37.829999999999984)  2.340010e+05   
183           184   (77.53000000000003, -38.71999999999997)  2.340020e+05   
185           186   (73.51300000000003, -53.10599999999994)  2.340100e+05   
187           188  (52.200000000000045, -46.42999999999995)  2.340300e+05   
188           189   (51.75000000000006, -46.41999999999996)  2.340400e+05   
...           ...                                       ...           ...   
1932333   1932334  (-67.43230081299998, 19.591019367000058)  1.774712e+09   
1932994   1932995   (-67.55183930699997, 19.60742207000004)  1.773141e+09   
1932995   1932996  (-67.50790808299996, 19.612875665000047)  1.779432e+09   
1932996   1932997  (-67.46447723199998, 19.627115288000027)  1.777859e+09   
1932997   1932998  (-67.42104638099994, 19.641354911000064)  1.774189e+09   

          latitude  longitude  HazardType  
182     -37.8

In [23]:
# Set output file path
output_table = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\unprocessed_data_table"

# Convert the DataFrame to a CSV file
unprocessed_data.to_csv(r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_unprocessed.csv", index=False)

# Use arcpy to load the CSV into ArcGIS Pro
arcpy.TableToTable_conversion(r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\ghz_unprocessed.csv", 
                              r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb", 
                              "unprocessed_data_table")

print(f"Table created at: {output_table}")

Table created at: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\unprocessed_data_table
